# Fannie Mae analysis

(just started)

This notebook contains some python code to analyse mortgage data
See the following link how to download data and [more details](https://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html).

Per quarter there is a Acquisition data set and a Performance data set. See the [details here](https://www.fanniemae.com/resources/file/fundmarket/pdf/webinar-101.pdf)


In [29]:
#### using the datatable package from h2o....  super package!
import pandas as pd
import numpy as np
import datatable as dt

## import acquisition and performance data

The data on mortgages is per quarter of starting mortgages. For example the file 2010Q1.txt contains all mortgaes that started in Q1 2010, each row is one mortgage.

The performance of the mortgage in the acquisition file are in the file Performance_2010Q1.txt. Multiple rows in this file correspond to one mortgage, For every mortgage we have the monthly performance of the mortgage, form its start until Dec of 2019.

In [55]:
%%time

#### import Acquisition data
acquisitions_Variables = [
    "LOAN_ID", "ORIG_CHN", "Seller_Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE",
    "FRST_DTE", "OLTV", "OCLTV", "NUM_BO", "Debt_to_Income", "Borrower_Credit_Score", "FTHB_FLG", "PURPOSE", "PROPERTY_TYPE",
    "NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", "MI_PCT", "Product_Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"
]

acquisition = dt.fread(
    "data/Acquisition_2010Q1.txt",
    sep = "|",
    header = None ,
    columns = acquisitions_Variables
)

acquisition = acquisition.to_pandas()
acquisition.shape

CPU times: user 3.53 s, sys: 213 ms, total: 3.74 s
Wall time: 768 ms


(323174, 25)

In [6]:
### five random five records
acquisition.sample(5)

,LOAN_ID,ORIG_CHN,Seller_Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,PROP_TYP,NUM_UNIT,OCC_STAT,STATE,ZIP_3,MI_PCT,Product_Type,CSCORE_C,MI_TYPE,RELOCATION_FLG
251264,800240213011,C,OTHER,4.875,45000,180,01/2010,03/2010,53,53.0,...,SF,1,P,NC,286,NaN,FRM,653.0,NaN,N
199278,655083009567,C,"WELLS FARGO BANK, N.A.",4.625,417000,360,12/2009,02/2010,52,52.0,...,CO,1,S,FL,341,NaN,FRM,793.0,NaN,N
26070,172441164363,B,SUNTRUST MORTGAGE INC.,4.250,95000,120,02/2010,04/2010,49,49.0,...,SF,1,P,KY,402,NaN,FRM,783.0,NaN,N
235617,756719413369,R,"BANK OF AMERICA, N.A.",4.875,89000,360,02/2010,04/2010,85,85.0,...,SF,1,P,WI,544,12.0,FRM,758.0,2.0,N
51789,243492275169,R,OTHER,5.250,132000,360,02/2010,04/2010,80,80.0,...,SF,1,P,MI,493,NaN,FRM,790.0,NaN,N


In [9]:
%%time

#### Import performance data
performance_Variables = [
    "LOAN_ID", "Monthly_Rpt_Prd", "Servicer_Name", "LAST_RT", "LAST_UPB", "Loan_Age", "Months_To_Legal_Mat",
    "Adj_Month_To_Mat", "Maturity_Date", "MSA", "Delq_Status", "MOD_FLAG", "Zero_Bal_Code", "ZB_DTE", "LPI_DTE",
    "Foreclosure_date","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS", "CE_PROCS", "RMW_PROCS",
    "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"
]

performance = dt.fread(
    "data/Performance_2010Q1.txt",
    sep = "|",
    header = None ,
    columns = performance_Variables
)

#performance = performance.to_pandas()
performance.shape

CPU times: user 30.7 s, sys: 6.11 s, total: 36.8 s
Wall time: 6.63 s


(18634553, 31)

In [11]:
#### first 5 records
performance.head(5)

,LOAN_ID,Monthly_Rpt_Prd,Servicer_Name,LAST_RT,LAST_UPB,Loan_Age,Months_To_Legal_Mat,Adj_Month_To_Mat,Maturity_Date,MSA,…,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪,▪▪▪▪
0,100010079393,02/01/2010,"WELLS FARGO BANK, N.A.",4.875,NA,0,360,360,02/2040,12420,…,NA,NA,,NA,N
1,100010079393,03/01/2010,,4.875,NA,1,359,358,02/2040,12420,…,NA,NA,,NA,N
2,100010079393,04/01/2010,,4.875,NA,2,358,358,02/2040,12420,…,NA,NA,,NA,N
3,100010079393,05/01/2010,,4.875,NA,3,357,357,02/2040,12420,…,NA,NA,,NA,N
4,100010079393,06/01/2010,,4.875,NA,4,356,355,02/2040,12420,…,NA,NA,,NA,N


## Start with a simple analysis

This will be the easiest in terms of data prep. Look only at mortgages starting in one specific quarter. For the performance we look at foreclosure or not. So if there is a date in the Foreclosure_date column then the mortgage defaulted otherwise the mortgae did not defaulted. This is the worst that can happen to a mortgage. Latwer we will look at a different performance, days past due where the mortgage does not neccesarily goes default.


In [20]:
foreclosures = performance[dt.f.Foreclosure_date != "", ["LOAN_ID", "Foreclosure_date"]]
foreclosures = foreclosures.to_pandas()

In [35]:
foreclosures

,LOAN_ID,Foreclosure_date
0,100446401716,11/01/2012
1,100683589832,11/01/2016
2,100960628290,03/01/2015
3,101212627627,05/01/2016
4,101455126320,12/01/2013
...,...,...
1669,997815231849,07/01/2019
1670,998148417849,06/01/2015
1671,998710529644,04/01/2015
1672,999261560724,11/01/2017


In [56]:
%%time

mortgages = (
    acquisition
    .merge(
        foreclosures,
        how="left",
        left_on="LOAN_ID",
        right_on="LOAN_ID"
    )
    .filter([ 
        "LOAN_ID","ORIG_DTE","FRST_DTE", "Debt_to_Income", "Borrower_Credit_Score", "PURPOSE",
        "Monthly_Rpt_Prd", "Loan_Age", "Seller_Name", "ORIG_RT",	"ORIG_AMT",
        "Zero_Bal_Code", "Delq_Status", "ZB_DTE", "LPI_DTE", "Foreclosure_date"
    ])
)

mortgages = (
    mortgages
    .assign(target = mortgages.Foreclosure_date.notna().astype(int))
)

CPU times: user 615 ms, sys: 139 ms, total: 754 ms
Wall time: 758 ms


In [57]:
mortgages

,LOAN_ID,ORIG_DTE,FRST_DTE,Debt_to_Income,Borrower_Credit_Score,PURPOSE,Seller_Name,ORIG_RT,ORIG_AMT,Foreclosure_date,target
0,100010079393,01/2010,03/2010,32.0,773.0,P,"WELLS FARGO BANK, N.A.",4.875,284000,NaN,0
1,100013622306,12/2009,02/2010,24.0,770.0,C,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,NaN,0
2,100019943199,11/2009,01/2010,21.0,806.0,P,OTHER,5.000,417000,NaN,0
3,100022098429,01/2010,03/2010,50.0,682.0,P,OTHER,5.250,461000,NaN,0
4,100023088745,11/2009,01/2010,39.0,804.0,P,"WELLS FARGO BANK, N.A.",5.250,100000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
323169,999990451380,12/2009,02/2010,35.0,731.0,P,"WELLS FARGO BANK, N.A.",4.875,520000,NaN,0
323170,999993511488,12/2009,02/2010,16.0,778.0,R,"WELLS FARGO BANK, N.A.",5.125,320000,NaN,0
323171,999993982336,12/2009,02/2010,35.0,743.0,R,"BANK OF AMERICA, N.A.",4.500,182000,NaN,0
323172,999998369629,02/2010,04/2010,33.0,683.0,C,OTHER,5.375,82000,NaN,0


In [51]:
mortgages.target.describe()

count    323174.000000
mean          0.005180
std           0.071785
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: target, dtype: float64

The column Delq_Status is the Loan Delinquency status and has the following meaning:

* 0 - "Current or less than 30 days past due"
* 1 - "30 - 59 days past due"
* 2 - "60 - 89 days past due"
* 3 - "90 - 119 days past due"
* 4 - "120 - 149 days past due"
* 5 - "150 - 179 days past due"
* 6 - "180 Day Delinquency"
* 7 - "210 Day Delinquency"
* 8 - "240 Day Delinquency"
* 9 - "270 Day Delinquency" / "270+ Day Delinquency"

In [22]:
test.Delq_Status.value_counts()

0     18135400
X       264341
1       127953
2        29554
3        13632
        ...   
99           1
98           1
95           1
96           1
97           1
Name: Delq_Status, Length: 102, dtype: int64

In [23]:
test.query("Delq_Status == '5'")

,LOAN_ID,ORIG_DTE,FRST_DTE,Monthly_Rpt_Prd,Loan_Age,Seller_Name,ORIG_RT,ORIG_AMT,Zero_Bal_Code,Delq_Status,ZB_DTE,LPI_DTE
1995,100091653134,12/2009,02/2010,07/01/2018,102,OTHER,5.375,312000,NaN,5,,
1996,100091653134,12/2009,02/2010,08/01/2018,103,OTHER,5.375,312000,NaN,5,,
1997,100091653134,12/2009,02/2010,09/01/2018,104,OTHER,5.375,312000,NaN,5,,
6564,100312066648,01/2010,03/2010,03/01/2014,49,OTHER,5.250,160000,NaN,5,,
8863,100446401716,12/2009,02/2010,09/01/2011,20,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",5.125,244000,NaN,5,,
...,...,...,...,...,...,...,...,...,...,...,...,...
18631508,999865000263,01/2010,03/2010,11/01/2013,45,OTHER,5.125,60000,NaN,5,,
18631519,999865000263,01/2010,03/2010,10/01/2014,56,OTHER,5.125,60000,NaN,5,,
18631539,999865000263,01/2010,03/2010,06/01/2016,76,OTHER,5.125,60000,NaN,5,,
18632769,999919586679,12/2009,02/2010,01/01/2017,84,"WELLS FARGO BANK, N.A.",5.000,427000,NaN,5,,


In [25]:
test.query("LOAN_ID == 100091653134")

,LOAN_ID,ORIG_DTE,FRST_DTE,Monthly_Rpt_Prd,Loan_Age,Seller_Name,ORIG_RT,ORIG_AMT,Zero_Bal_Code,Delq_Status,ZB_DTE,LPI_DTE
1893,100091653134,12/2009,02/2010,01/01/2010,0,OTHER,5.375,312000,NaN,0,,
1894,100091653134,12/2009,02/2010,02/01/2010,1,OTHER,5.375,312000,NaN,0,,
1895,100091653134,12/2009,02/2010,03/01/2010,2,OTHER,5.375,312000,NaN,0,,
1896,100091653134,12/2009,02/2010,04/01/2010,3,OTHER,5.375,312000,NaN,0,,
1897,100091653134,12/2009,02/2010,05/01/2010,4,OTHER,5.375,312000,NaN,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
2008,100091653134,12/2009,02/2010,08/01/2019,115,OTHER,5.375,312000,NaN,0,,
2009,100091653134,12/2009,02/2010,09/01/2019,116,OTHER,5.375,312000,NaN,1,,
2010,100091653134,12/2009,02/2010,10/01/2019,117,OTHER,5.375,312000,NaN,1,,
2011,100091653134,12/2009,02/2010,11/01/2019,118,OTHER,5.375,312000,NaN,1,,
